# Fitting First Order ODE Solutions to Given Data


## Solving First Order ODE

Let $y(t)$ be defined by a system of first order ODEs which involve some unknown parameters $p = (p_1, \ldots, p_l)$. That is

$$\frac{dy(t)}{dt} = f(y(t), t, p)$$

Here, $y$ can be an $\mathbb{R}$-valued or vector-valued function. If $y(t) = (y_1(t), \ldots, y_k(t))$, then $\frac{dy(t)}{dt} = (\frac{dy_1(t)}{dt}, \ldots, \frac{dy_k(t)}{dt})$.